# Pre-processing the Lite Yelp Review Dataset

As presented in the NLP with PyTorch book

In [2]:
# import libraries
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [3]:
args = Namespace(
    raw_train_dataset_csv="data/yelp/raw_train.csv",
    raw_test_dataset_csv="data/yelp/raw_test.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="data/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [7]:
# Read raw data
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])

In [15]:
# making the subset equal across the review classes
by_rating = collections.defaultdict(list)
# print(by_rating)

for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())

review_subset = []

for _, item_list in sorted(by_rating.items()):
    n_total = len(item_list)
    n_subset = int(args.proportion_subset_of_train * n_total)
    review_subset.extend(item_list[:n_subset])

review_subset = pd.DataFrame(review_subset)
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [16]:
train_reviews.rating.value_counts()

1    280000
2    280000
Name: rating, dtype: int64

In [17]:
review_subset.rating.value_counts()

1    28000
2    28000
Name: rating, dtype: int64

In [18]:
# unique classes
set(review_subset.rating)

{1, 2}

In [19]:
# splitting the subset by rating to create our new train, val, and test splits
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

final_list = []

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)

    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'

    for item in item_list[n_train : n_train + n_val]:
        item['split'] = 'val'

    for item in item_list[n_train + n_val : n_train + n_val + n_test]:
        item['split'] = 'test'

    # Add to final list
    final_list.extend(item_list)


In [20]:
final_reviews = pd.DataFrame(final_list)

In [21]:
final_reviews.split.value_counts()

train    39200
val       8400
test      8400
Name: split, dtype: int64

In [23]:
# Preprocess the reviews
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

In [24]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [25]:
final_reviews.head()

,rating,review,split
0,negative,"i will never go back ! the food is good , but ...",train
1,negative,this place was below average . specializing in...,train
2,negative,this is my second trip to gino s . first trip ...,train
3,negative,never again . i ve given t tom s two tries bec...,train
4,negative,"this is really sad , as osaka used to be relat...",train


In [26]:
final_reviews.to_csv(args.output_munged_csv, index=False)